In [1]:
import pandas as pd
import numpy as np
import re

### Trying to read the 'movie_titles.csv' gives an error.

In [11]:
file = './data/movie_titles.txt'

In [12]:
movie_titles = pd.read_csv(file)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 72, saw 4


### Looking at line 72, we see that there are commas in titles which confuses the csv reader. Hence the error

In [15]:
!sed '72q;d' './data/movie_titles.txt'  

72,1974,At Home Among Strangers, A Stranger Among His Own


### The idea now is to find a pattern in commas that we can exploit and change the seperator from comma to something else (I chose semicolon). First we read the file using standard python open() method and check if the number of lines we get are the same as in the file

In [18]:
with open(file,'r',encoding='utf-8',errors='ignore') as f:
    data = f.readlines()

In [19]:
!wc -l ./data/movie_titles.txt

17770 ./data/movie_titles.txt


In [20]:
len(data)

17770

### Reading like above adds the newline character to every line. Removing it using strip() method of strings, we get the data as a list. (strip() drops any white space from beginning or end of a string)

In [21]:
data = [d.strip() for d in data]

In [22]:
data[:5]

['1,2003,Dinosaur Planet',
 '2,2004,Isle of Man TT 2004 Review',
 '3,1997,Character',
 "4,1994,Paula Abdul's Get Up & Dance",
 '5,2004,The Rise and Fall of ECW']

### First we check that our chosen seperator is not in the dataset. If it is, we cannot use it. 

In [23]:
for d in data:
    f=re.search(';',d)
    if f is not None:
        print(f)

### Looking at the five lines, we can see that the first two commas in every line are good candidates for being a seperator, so we use python's regex to replace first 2 commas to semicolon like so:

In [24]:
new_data=[]
for d in data:
    new_data.append(re.sub(',',';', d,2))

In [25]:
new_data[70:75]

['71;1995;Maya Lin: A Strong Clear Vision',
 '72;1974;At Home Among Strangers, A Stranger Among His Own',
 '73;1954;Davy Crockett: 50th Anniversary Double Feature',
 '74;1999;6ixtynin9',
 '75;1997;Grind']

In [26]:
with open('./data/movie_titles_tmp.csv','w') as f:
    for d in new_data:
        f.write(d+'\n')

### Now we can try to read the dataset again, using semicolon as seperator

In [27]:
df=pd.read_csv('./data/movie_titles_tmp.csv',sep=';',header=None,names=['movie_id','year','title'],index_col='movie_id')

In [28]:
df.head()

,year,title
movie_id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW


### We want to convert the year from float to int. But we get an error

In [29]:
df.year=df.year.astype('int32')

ValueError: Cannot convert non-finite values (NA or inf) to integer

### Apparently there is an NA or Inf and python cannot convert to integer. So we check the unique values of year column

In [30]:
df.year.unique()

array([2003., 2004., 1997., 1994., 1992., 1991., 2001., 1999., 1947.,
       1982., 1988., 1996., 2005., 2000., 1972., 2002., 1981., 1962.,
       1973., 1964., 1952., 1941., 1995., 1969., 1943., 1989., 1974.,
       1954., 1956., 1979., 1951., 1983., 1998., 1985., 1965., 1993.,
       1976., 1957., 1987., 1927., 1978., 1975., 1971., 1939., 1968.,
       1980., 1950., 1921., 1918., 1942., 1959., 1958., 1966., 1935.,
       1967., 1948., 1990., 1984., 1937., 1986., 1916., 1946., 1970.,
       1963., 1961., 1931., 1960., 1955., 1940., 1977., 1953., 1944.,
       1928., 1945., 1930., 1936., 1938., 1929., 1932., 1949., 1933.,
       1925., 1922., 1924., 1934., 1920., 1915., 1926., 1917.,   nan,
       1909., 1919., 1923., 1896., 1914.])

### We see that there is a 'nan'. Then we try to filter out in the dataset and find a handful columns.

In [31]:
df[df.year.isna()]

,year,title
movie_id,,
4388,NaN,Ancient Civilizations: Rome and Pompeii
4794,NaN,Ancient Civilizations: Land of the Pharaohs
7241,NaN,Ancient Civilizations: Athens and Greece
10782,NaN,Roti Kapada Aur Makaan
15918,NaN,Hote Hote Pyaar Ho Gaya
16678,NaN,Jimmy Hollywood
17667,NaN,Eros Dance Dhamaka


### For now we just drop the columns. First we find the indices of these columns and then use pandas drop() method to delete them

In [32]:
year_na = df[df.year.isna()].index
df.drop(year_na,axis=0,inplace=True)

### Finally we check again that there is no nan in the year column

In [33]:
df[df.year.isna()]

,year,title
movie_id,,


In [34]:
df.year=df.year.astype('int32')

In [35]:
df.head()

,year,title
movie_id,,
1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW


### Finally we save the clean dataset

In [36]:
df.to_csv('./data/movie_titles_clean.csv')